In [ ]:
from EPA import *
from EPA import encode_key,model_build, fit_eval, anomaly_match, trace_seq_path, train_batch
#!jupyter notebook list

## Log key execuition path Anomaly

### Model architecture 

 - Log key execution path anomaly
 - LSTM with history 5
 
**Training**:
 - set the path_filename
 - set the log_value_vector for Training
 
**Testing**:
 - set the path_filename
 - set the log_value_vector

### The principles behind Execution path anomaly
1. first, we generate the set of distinct log keys from our program
2. parse the entries into log keys, the log key sequence ---- an execution path
3. the model DeepLog is a multi-class classifier over recent context

1. input the recent log keys
2. a probability distribution over the n log keys from K ---- the probability that the next log key in the sequence is a key ki belongs to K

### How LSTM Works

- each block remembers a state for its input as a vector of a fixed dimension

- input: a window w of h log keys --- (w = {mt−h, . . . ,mt−1})
- output: the log key value comes right after w

- the loss function will be categorical cross-entropy loss

- omits the input layer and and output layer ---- encoding-decoding schemes
- the input layer encodes the n possible log keys from K(log keys set) as one-hot vectors

- output layer:  a standard multinomial logistic function to represent **Pr[mt = ki|w]**
"""


In [ ]:
#df = pd.read_csv(f'data/log_value_vector.csv')
df = pd.read_pickle(f'data/log_value_vector.pkl')
#df = df.iloc[df.index < len(df.index)* 0.2]
df

#### Step 1: Encode log key Sequences (one hot key)

In [ ]:
# read the data
encoder_file = PurePosixPath('encoder.save')
model_file = PurePosixPath('model.h5')
X, one_hot_y, class_num = encode_key(df['log key'], encoder_file, 5)

X.shape, one_hot_y.shape  # seq -> next

#### Step 2: LSTM Model (train, test)

In [ ]:
%time model = model_build(5, class_num)

In [ ]:
%time classifier = fit_eval(model, model_file ,30, X, one_hot_y)

In [ ]:
%time seq_pre_dict = anomaly_match(classifier, X, one_hot_y, 0.0001) # for initial learned anomalies 0.001 is better
trace_df = df.iloc[:len(X),]
#seq_pre_dict = anomaly_match(classifier, X, one_hot_y, 0.01)
#trace_df = df.iloc[:,]

In [ ]:
#for key, value in seq_pre_dict.items():
#    print(key)
#    print(len(value))
#print(len(trace_df))

In [ ]:
new_trace_df = trace_seq_path(trace_df, seq_pre_dict)
new_trace_df.to_csv(PurePosixPath('EPA_trace_df.csv'))
print(new_trace_df['path_pred'].value_counts())
new_trace_df.head(50)

#### Step 3: `Online learning` 

In [ ]:
#train_batch(classifier, X[:100], one_hot_y[:100])
%time model2 = train_batch(classifier,model_file,  X[:100], one_hot_y[:100], 0.0001,5) # 0.05 probab is better to learn 